In [ ]:
# ==========================================================
# Healthcare Predictive Analytics – Patient Readmission Prediction
# Tools: Python, Pandas, Scikit-Learn
# ==========================================================

# Step 1: Import Libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# Step 2: Load Dataset
data = pd.read_csv("PP1Healthcare.csv")

# Step 3: Data Cleaning
data.columns = data.columns.str.strip().str.replace(" ", "_")
data = data.dropna()

# Step 4: Feature Engineering
# Create a synthetic target: 'Readmitted' (Yes/No) based on certain risk factors
np.random.seed(42)
data["Readmitted"] = np.where(
    (data["Medical_Condition"].isin(["Diabetes", "Cancer"])) |
    (data["Age"] > 60) |
    (data["Billing_Amount"] > data["Billing_Amount"].median()),
    "Yes", "No"
)

# Step 5: Encode Categorical Variables
le = LabelEncoder()
for col in data.select_dtypes(include=['object']).columns:
    data[col] = le.fit_transform(data[col])

# Step 6: Define Features & Target
X = data.drop("Readmitted", axis=1)
y = data["Readmitted"]

# Step 7: Split Data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

# Step 8: Feature Scaling
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Step 9: Model 1 – Logistic Regression
lr_model = LogisticRegression(max_iter=1000)
lr_model.fit(X_train_scaled, y_train)
lr_pred = lr_model.predict(X_test_scaled)
lr_acc = accuracy_score(y_test, lr_pred)

# Step 10: Model 2 – Random Forest
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)
rf_pred = rf_model.predict(X_test)
rf_acc = accuracy_score(y_test, rf_pred)

# Step 11: Display Results
print("=== Logistic Regression Results ===")
print("Accuracy:", round(lr_acc*100, 2), "%")
print(classification_report(y_test, lr_pred))

print("\n=== Random Forest Results ===")
print("Accuracy:", round(rf_acc*100, 2), "%")
print(classification_report(y_test, rf_pred))

# Step 12: Confusion Matrices
print("\nLogistic Regression Confusion Matrix:\n", confusion_matrix(y_test, lr_pred))
print("\nRandom Forest Confusion Matrix:\n", confusion_matrix(y_test, rf_pred))

# Step 13: Export Results for Power BI
results = pd.DataFrame({
    "Actual": y_test,
    "Predicted_RF": rf_pred,
    "Predicted_LR": lr_pred
})
results.to_csv("Healthcare_Readmission_Results.csv", index=False)

print("\n✅ Exported 'Healthcare_Readmission_Results.csv' for Power BI dashboard.")